In [1]:
from rdkit.Chem import AllChem
from rdkit.Chem import rdChemReactions
from rdkit import Chem
from rdkit.Chem import Draw, AllChem
from rdkit.Chem.Descriptors import NumRadicalElectrons
from IPython.display import Image, display
from enum_rxns import *
import pandas as pd
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

In [2]:
rxn_sma_cyc_3mem = '[C;X3v3+0:1]-[*:2]-[O:3]-[OH:4]>>[C;X3v3+0:1]1[*:2]-[O:3]1.[OH:4]'
rxn_sma_cyc_4mem = '[C;X3v3+0:1]-[*:2]-[*:3]-[O:4]-[OH:5]>>[C;X3v3+0:1]1[*:2]-[*:3]-[O:4]1.[OH:5]'
rxn_sma_cyc_5mem = '[C;X3v3+0:1]-[*:2]-[*:3]-[*:4]-[O:5]-[OH:6]>>[C;X3v3+0:1]1[*:2]-[*:3]-[*:4]-[O:5]1.[OH:6]'
rxn_sma_cyc_6mem = '[C;X3v3+0:1]-[*:2]-[*:3]-[*:4]-[*:5]-[O:6]-[OH:7]>>[C;X3v3+0:1]1[*:2]-[*:3]-[*:4]-[*:5]-[O:6]1.[OH:7]'
beta_scis = '[C;X3v3+0:1]-[*:2]-[*:3]>>[C:1]=[*:2].[*:3]'

rxns = [rxn_sma_cyc_3mem, rxn_sma_cyc_4mem, rxn_sma_cyc_5mem, rxn_sma_cyc_6mem,
       beta_scis]
rxns_str = ['cyc_3mem', 'cyc_4mem', 'cyc_5mem', 'cyc_6mem', 'beta_scis']


df = pd.concat([pd.read_csv('ro2_qooh_'+ str(x) + 'mem.csv') for x in [5,6,7,8]])

#print(len(df), len(df.P.unique()))
#print(len(set([Chem.MolToSmiles(Chem.MolFromSmiles(x)) for x in df.P])))
reactants = [[x] for x in list(df.P)] # QOOH's
#print(len(reactants))

df_all = pd.DataFrame(columns = ['rxn_smiles','R','P','rxn_type'])

Images = []
rxn_smiles_all = []
for a, rxn_to_run in enumerate(rxns):
    for i, reactant in enumerate(reactants):
        possible_products = run_rxn(rxn_to_run, reactant)
        #if possible_products == []:
        #    print(reactant)
        #print(possible_products)

        for j, prod in enumerate(possible_products):
            if '[HH]' in prod:
                continue
            rxn_smiles = '.'.join(reactant)+'>>'+'.'.join(prod)
            #print(rxn_smiles, rxn_type)
            rxn_smiles_all.append(rxn_smiles)
            #png = visualize_rxn(rxn_smiles, 'test'+str(i)+str(j)+'.png')
            #Images.append(Image(filename = 'test'+str(i)+str(j)+'.png'))
            #IPython.display.Image(png)
        
    new_df = pd.DataFrame(list(map(list, zip(*[rxn_smiles_all]))), columns = ['rxn_smiles'])
    new_df['R'] = [ x.split('>>')[0] for x in new_df.rxn_smiles]
    new_df['P'] = [ x.split('>>')[1] for x in new_df.rxn_smiles]
    new_df['rxn_type'] = [rxns_str[a]] * len(new_df)
    df_all = pd.concat([df_all, new_df])

In [3]:
#olefin
olefin_formation = '[O:1]-[O:2]-[C:3]-[C:4]-[H:5]>>[O:1]-[O:2]-[H:5].[C:3]=[C:4]'
reactants = [[x] for x in list(df.R.unique())] # RO2's

rxn_smiles_all = []
for i, reactant in enumerate(reactants):
    possible_products = run_rxn(olefin_formation, reactant)
    #print(possible_products)

    for j, prod in enumerate(possible_products):
        if '[HH]' in prod:
            continue
        rxn_smiles = '.'.join(reactant)+'>>'+'.'.join(prod)
        #print(rxn_smiles, rxn_type)
        rxn_smiles_all.append(rxn_smiles)
        #png = visualize_rxn(rxn_smiles, 'test'+str(i)+str(j)+'.png')
        #Images.append(Image(filename = 'test'+str(i)+str(j)+'.png'))
        #IPython.display.Image(png)
        
new_df = pd.DataFrame(list(map(list, zip(*[rxn_smiles_all]))), columns = ['rxn_smiles'])
new_df['R'] = [ x.split('>>')[0] for x in new_df.rxn_smiles]
new_df['P'] = [ x.split('>>')[1] for x in new_df.rxn_smiles]
new_df['rxn_type'] = ['olefin_formation'] * len(new_df)
df_all = pd.concat([df_all, new_df])

In [4]:
#qooh to o2qooh
qooh_to_o2qooh = '[C;X3v3+0:1].[O:2]=[O:3]>>[C:1]-[O:2]-[O:3]'
reactants = [[x, 'O=O'] for x in list(df.P)] # QOOH's

rxn_smiles_all = []
for i, reactant in enumerate(reactants):
    possible_products = run_rxn(qooh_to_o2qooh, reactant)
    #print(possible_products)

    for j, prod in enumerate(possible_products):
        if '[HH]' in prod:
            continue
        rxn_smiles = '.'.join(reactant)+'>>'+'.'.join(prod)
        rxn_smiles_all.append(rxn_smiles)

In [5]:
new_df = pd.DataFrame(list(map(list, zip(*[rxn_smiles_all]))), columns = ['rxn_smiles'])
new_df['R'] = [ x.split('>>')[0] for x in new_df.rxn_smiles]
new_df['P'] = [ x.split('>>')[1] for x in new_df.rxn_smiles]
new_df['rxn_type'] = ['qooh_to_o2qooh'] * len(new_df)
df_all = pd.concat([df_all, new_df])

In [6]:
#o2qooh to khp
o2qooh_to_khp_H_5mem = '[O:1]-[O:2]-[*:3]-[*:4](-[H:7])-[O:5]-[OH:6]>>[H:7]-[O:1]-[O:2]-[*:3]-[*:4]=[O:5].[OH:6]'
o2qooh_to_khp_H_6mem = '[O:1]-[O:2]-[*:3]-[*:4]-[*:8](-[H:7])-[O:5]-[OH:6]>>[H:7]-[O:1]-[O:2]-[*:3]-[*:4]-[*:8]=[O:5].[OH:6]'
o2qooh_to_khp_H_7mem = '[O:1]-[O:2]-[*:3]-[*:4]-[*:8]-[*:9](-[H:7])-[O:5]-[OH:6]>>[H:7]-[O:1]-[O:2]-[*:3]-[*:4]-[*:8]-[*:9]=[O:5].[OH:6]'
o2qooh_to_khp_H_8mem = '[O:1]-[O:2]-[*:3]-[*:4]-[*:8]-[*:9]-[*:10](-[H:7])-[O:5]-[OH:6]>>[H:7]-[O:1]-[O:2]-[*:3]-[*:4]-[*:8]-[*:9]-[*:10]=[O:5].[OH:6]'

rxn_smiles_all = []
reactants = [  [x] for x in list(df_all[df_all.rxn_type == 'qooh_to_o2qooh'].P)]
for i, reactant in enumerate(reactants):
    possible_products = run_rxn(o2qooh_to_khp_H_5mem, reactant)
    possible_products += run_rxn(o2qooh_to_khp_H_6mem, reactant)
    possible_products += run_rxn(o2qooh_to_khp_H_7mem, reactant)
    possible_products += run_rxn(o2qooh_to_khp_H_8mem, reactant)

    #if possible_products == []:
    #    print(reactant[0])
    
    for j, prod in enumerate(possible_products):
        rxn_smiles = '.'.join(reactant)+'>>'+'.'.join(prod)
        rxn_smiles_all.append(rxn_smiles)
        
new_df = pd.DataFrame(list(map(list, zip(*[rxn_smiles_all]))), columns = ['rxn_smiles'])
new_df['R'] = [ x.split('>>')[0] for x in new_df.rxn_smiles]
new_df['P'] = [ x.split('>>')[1] for x in new_df.rxn_smiles]
new_df['rxn_type'] = ['KHP_formation'] * len(new_df)
df_all = pd.concat([df_all, new_df])


In [7]:
df_all.to_csv('qooh_to_cyc+bscis+olefin+o2qooh+khp.csv', index = False)

In [8]:
df.to_csv('ro2_qooh_all.csv', index = False)